In [47]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate
sym.init_printing(use_unicode=True)

a.

In [48]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn
    
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

def GetHermite(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = (2*x)*GetHermite(n-1,x)-(2*(n-1))*GetHermite(n-2,x)

    return sym.expand(poly,x)

def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

def GetAllRootsGHer(n):
    
    xn = np.linspace((-1)*((4*n)**(1/2)),(4*n)**(1/2),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    Hermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n-1],'numpy')
    Weights = ((2**(n-1))*(np.math.factorial(n))*(np.pi**(1/2)))/((n**2)*((poly(Roots)**2)))
    
    return Weights

n = 20
raices = GetAllRootsGHer(n)
pesos = GetWeightsGHer(n)
print(raices)
print(pesos)


[-5.38748089 -4.60368245 -3.94476404 -3.34785457 -2.78880606 -2.254974
 -1.73853771 -1.23407622 -0.73747373 -0.24534071  0.24534071  0.73747373
  1.23407622  1.73853771  2.254974    2.78880606  3.34785457  3.94476404
  4.60368245  5.38748089]
[2.22939365e-13 4.39934099e-10 1.08606937e-07 7.80255648e-06
 2.28338636e-04 3.24377334e-03 2.48105209e-02 1.09017206e-01
 2.86675505e-01 4.62243670e-01 4.62243670e-01 2.86675505e-01
 1.09017206e-01 2.48105209e-02 3.24377334e-03 2.28338636e-04
 7.80255648e-06 1.08606937e-07 4.39934099e-10 2.22939365e-13]


b.

In [49]:
def estado_del_oscilador_al_cuadrado(x,n=1):
    f = (1/((2**n) * np.math.factorial(n)))*((1/np.pi)*(1/2))*4*(x**2)
    return f

def posicion(x):
    p = abs((estado_del_oscilador_al_cuadrado(x)))*(x**2)
    return p

Roots, Weights = np.polynomial.hermite.hermgauss(6)
I = np.sum(posicion(Roots)*Weights)

f2 = lambda x: posicion(x)* np.exp(-x**2)
exact = integrate.quad(f2,-np.inf,np.inf)
print(exact)
print(I)

(0.4231421876608171, 5.0481339624845845e-09)
0.4231421876608168
